In [2]:
import re
import os
import numpy as np
import pandas as pd

from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
sns.set_context("notebook", font_scale=1)
matplotlib.style.use('ggplot')
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

/Users/bmigliozzi/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
hdAct1 = 'AAAAGACATCTTTTCCAAATCAGTCAGTTCCAGCACCGACGCCACCCGGATTCCCCCGTCATCGCATCGCTGTTCGCTATTCATCCCTGCGCTATTTTTCACTTTAGTTTCGCGTAATCCCTCAGAACAGTCGCAATGGAAGACGAAGTTGCCGCCTTGGTCGTGGACAATGGATCCGGTATGTGCAAGGCCGGATTTGCCGGAGATGACGCTCCCCGCGCCGTCTTCCCCTCCATCGTTGGCCGACCCCGTCATCAGGGTGTCATGGTCGGTATGGGTCAAAAGGACAGCTACGTCGGTGATGAGGCCCAGAGCAAGCGCGGTATCCTGACGCTCAAGTACCCCATCGAGCACGGCATCGTCACCAACTGGGATGACATGGAGAAGATCTGGCATCACACCTTCTACAACGAGCTCCGCGTGGCTCCCGAGGAACACCCCGTCCTCCTGACTGAGGCTCCCCTCAACCCCAAGGCCAACAGGGAAAAGATGACCCAGATCATGTTCGAGACATTCAACACCCCCGCCATGTACGTCGCGATTCAGGCTGTGCTCTCCCTGTACGCGTCCGGTCGTACCACCGGTATCGTGCTGGACTCTGGTGATGGTGTCTCCCACACTGTCCCCATCTACGAAGGTTATGCTCTGCCTCACGCCATCCTCCGTCTGGATTTGGCCGGTCGCGACTTGACTGACTACTTGATGAAGATCCTGACTGAGCGCGGTACAGCTTCGTCACAACGCT'
hdMag1 = 'CCCATTAGAGGGATTTCATCTATTTCAACTCTTGGGCATTTTCTCTGACGCTTAAATTTTTCCCAGACAATCTCGAAGGAAAACGAAAATGACGGAGGACCGTTTCTACGTGCGATACTACGTCGGCCATAAAGGCAAGTTCGGGCATGAATTTTTGGAATTCGAATTTCGCCCGGACGGTCGCCTGCGTTACGCCAACAATTCCAACTACAAGAACGACACGATGATCCGCAAGGAGGTGGTCGTCCATCCGGCTATCTTGGAAGAGGTGAAGCGCATCATTCAGGACAGTGAGATTCTCCGGGAGAACGACGCCAAGTGGCCGCAGCCGGACCGCGTGGGCCGGCAGGAGTTAGAGATTCTGCTGGACGACGAGCACATCTCCTTCAACACGGGAAAGATCGGCTCGCTGATGGACGTCAACAACAGCCCCGATCCAGAAGGTCTGCGTTGCTTCTACTACCTCGTGCAGGATCTCAAGTGTCTCGTCTTCTCCCTCATTGCGCTGCACTTTAAAATCAAGCCGATTTGAACGCTTGTAATCAACCAATCAAGCGCGAAGATTTGCACTAGCACCCTCGTCGAC'

In [25]:
%%time 
df =  pd.read_csv('pam_sites.csv')

CPU times: user 302 ms, sys: 48.7 ms, total: 351 ms
Wall time: 584 ms


In [26]:
df.head()

,Contig,Start,End,gRNA_Seq,PAM,Strand,G_start,Unique
0,nHd.2.3.scaf00001,2,21,GGTGACTGTATTCCGGGGGG,AGG,+,True,No
1,nHd.2.3.scaf00001,2875,2894,GAATGCATTTCGTTGAATGG,AGG,+,True,No
2,nHd.2.3.scaf00001,3222,3241,GAATGCATTTCGCTGAATGG,AGG,+,True,No
3,nHd.2.3.scaf00001,3291,3310,GCGGGCATCTCATTGTCGGG,AGG,+,True,Yes
4,nHd.2.3.scaf00001,3594,3613,GCATGCGCTGTTTCCAACGG,AGG,+,True,No


In [27]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Start,239950,38268.9,51313.5,1,6532,20655.5,49273.8,594043
End,239950,38268.9,51313.5,4,6530,20660.5,49270.8,594024
G_start,239950,1,0,True,1,1,1,True


In [39]:
def pairDNA(x):
    if(x=='A'):
        return 'T'
    elif(x=='T'):
        return 'A'
    elif(x=='C'):
        return 'G'
    else:
        return 'C'

def dnaComp(s):
    # Gets complementary strand of DNA
    return ''.join(map(lambda b: pairDNA(b), s))

In [76]:
df['gRNA_Seq_Comp'] = map(lambda x: dnaComp(x), df['gRNA_Seq'])

In [98]:
scaf209_df = df[df.Contig=='nHd.2.3.scaf00209']
display(scaf209_df.describe().T)
scaf20_df = df[df.Contig=='nHd.2.3.scaf00020']
display(scaf20_df.describe().T)

,count,mean,std,min,25%,50%,75%,max
Start,223,50446.3,28581.9,1953,25853,56215,73148,100028
End,223,50445.6,28583,1934,25853,56234,73129,100047
G_start,223,1,0,True,1,1,1,True


,count,mean,std,min,25%,50%,75%,max
Start,575,130381,70488.5,593,74255,138397,192537,252622
End,575,130382,70490.4,574,74255,138416,192518,252603
G_start,575,1,0,True,1,1,1,True


In [84]:
display(scaf209_df.sample(5))
display(scaf20_df.sample(5))

,Contig,Start,End,gRNA_Seq,PAM,Strand,G_start,Unique,gRNA_Seq_Comp
60501,nHd.2.3.scaf00209,44951,44970,GCTTGCTTAGTACCTCCCGG,CGG,+,True,No,CGAACGAATCATGGAGGGCC
60543,nHd.2.3.scaf00209,79241,79260,GAACAGTGGTGCCAGTAAGG,CGG,+,True,Yes,CTTGTCACCACGGTCATTCC
60497,nHd.2.3.scaf00209,42928,42947,GTATGAAATTGCCAGAGTGG,CGG,+,True,Yes,CATACTTTAACGGTCTCACC
60553,nHd.2.3.scaf00209,79470,79489,GGAGGAGGAGGAGGAGTGGG,AGG,+,True,No,CCTCCTCCTCCTCCTCACCC
60661,nHd.2.3.scaf00209,78941,78922,GGTGGAGGCGAGGAGCTCGG,AGG,-,True,Yes,CCACCTCCGCTCCTCGAGCC


,Contig,Start,End,gRNA_Seq,PAM,Strand,G_start,Unique,gRNA_Seq_Comp
11553,nHd.2.3.scaf00020,138012,137993,GTTCGGTGAGGACGTCGAGG,AGG,-,True,Yes,CAAGCCACTCCTGCAGCTCC
11144,nHd.2.3.scaf00020,45448,45467,GCATTTCAGGGTGACGCTGG,CGG,+,True,Yes,CGTAAAGTCCCACTGCGACC
11246,nHd.2.3.scaf00020,143517,143536,GTCATGCGTCCTGCCCCCGG,CGG,+,True,Yes,CAGTACGCAGGACGGGGGCC
11508,nHd.2.3.scaf00020,92023,92004,GCCCATTATGGATGACCCGG,AGG,-,True,Yes,CGGGTAATACCTACTGGGCC
11509,nHd.2.3.scaf00020,92554,92535,GCATTACTAAGACGTATGGG,TGG,-,True,No,CGTAATGATTCTGCATACCC


In [5]:
def matchSeq(gRNA,gene):
    match = re.findall(gRNA,gene)
    if(len(match)>0):
        return True
    else:
        return False

## Total Matches

In [114]:
scaf209_df['Match_gRNA'] = map(lambda x: matchSeq(x,hdAct1),scaf209_df['gRNA_Seq'])
scaf20_df['Match_gRNA'] = map(lambda x: matchSeq(x,hdMag1),scaf20_df['gRNA_Seq'])

print(len(scaf209_df[scaf209_df['Match_gRNA']==True]))
print(len(scaf20_df[scaf20_df['Match_gRNA']==True]))

0
0


## Unique Matches

In [105]:
uniq_df = df[df.Unique=='Yes']
uniq_df.describe().T

uniq_scaf209_df = uniq_df[uniq_df.Contig=='nHd.2.3.scaf00209']
display(uniq_scaf209_df.describe().T)
uniq_scaf20_df = uniq_df[uniq_df.Contig=='nHd.2.3.scaf00020']
display(uniq_scaf20_df.describe().T)

,count,mean,std,min,25%,50%,75%,max
Start,160,48657.5,28897.3,1953,23643.8,47753.5,73145.5,99529
End,160,48655.4,28895.9,1934,23624.8,47753.5,73126.5,99510
G_start,160,1,0,True,1,1,1,True


,count,mean,std,min,25%,50%,75%,max
Start,430,124202,69213,593,73514.8,118231,189411,252622
End,430,124202,69214,574,73495.8,118250,189402,252603
G_start,430,1,0,True,1,1,1,True


In [115]:
uniq_scaf209_df['Match_gRNA'] = map(lambda x: matchSeq(x,hdAct1),uniq_scaf209_df['gRNA_Seq'])
uniq_scaf20_df['Match_gRNA'] = map(lambda x: matchSeq(x,hdMag1),uniq_scaf20_df['gRNA_Seq'])

print(len(uniq_scaf209_df[uniq_scaf209_df['Match_gRNA']==True]))
print(len(uniq_scaf20_df[uniq_scaf20_df['Match_gRNA']==True]))

0
0


In [118]:
uniq_scaf209_df['Match_gRNA'] = map(lambda x: matchSeq(x,hdAct1),uniq_scaf209_df['gRNA_Seq_Comp'])
uniq_scaf20_df['Match_gRNA'] = map(lambda x: matchSeq(x,hdMag1),uniq_scaf20_df['gRNA_Seq_Comp'])

print(len(uniq_scaf209_df[uniq_scaf209_df['Match_gRNA']==True]))
print(len(uniq_scaf20_df[uniq_scaf20_df['Match_gRNA']==True]))

0
0


In [109]:
# uniq_scaf209_df.to_csv()
# uniq_scaf20_df.to_csv()

In [119]:
scaf209_df[scaf209_df['PAM']=='CGG']

,Contig,Start,End,gRNA_Seq,PAM,Strand,G_start,Unique,gRNA_Seq_Comp,Match_gRNA
60462,nHd.2.3.scaf00209,3573,3592,GTGAAACGTCGGGATGACGG,CGG,+,True,Yes,CACTTTGCAGCCCTACTGCC,False
60465,nHd.2.3.scaf00209,6249,6268,GACGGGCACAACACACGGGG,CGG,+,True,Yes,CTGCCCGTGTTGTGTGCCCC,False
60467,nHd.2.3.scaf00209,9278,9297,GTCGGTTTGGTACAGCTGGG,CGG,+,True,Yes,CAGCCAAACCATGTCGACCC,False
60468,nHd.2.3.scaf00209,11741,11760,GCGCCCGATACCCCCGTCGG,CGG,+,True,Yes,CGCGGGCTATGGGGGCAGCC,False
60469,nHd.2.3.scaf00209,12700,12719,GCGCCCGATACTCCCGTCGG,CGG,+,True,Yes,CGCGGGCTATGAGGGCAGCC,False
60470,nHd.2.3.scaf00209,15285,15304,GGCGCACGCCCATTGGTGGG,CGG,+,True,Yes,CCGCGTGCGGGTAACCACCC,False
60480,nHd.2.3.scaf00209,25686,25705,GCAAGCGGAAGATATAACGG,CGG,+,True,Yes,CGTTCGCCTTCTATATTGCC,False
60482,nHd.2.3.scaf00209,28982,29001,GGGTTGAATCACCAAAGTGG,CGG,+,True,Yes,CCCAACTTAGTGGTTTCACC,False
60485,nHd.2.3.scaf00209,30287,30306,GATGCACACAGAATATTTGG,CGG,+,True,Yes,CTACGTGTGTCTTATAAACC,False
60486,nHd.2.3.scaf00209,31605,31624,GCCCTTGAATCAGGTGCAGG,CGG,+,True,Yes,CGGGAACTTAGTCCACGTCC,False


In [39]:
!ngg2 --outputFile scaf209_pam_sites.csv nHd.2.3.scaf00209
# !ngg2 --outputFile scaf209_pam_sites.csv --allowNoncanonical nHd.2.3.scaf00209

2016-09-12 11:22:33,966 - ngg2 - INFO - ngg2 vv1.3.0

Options
FASTA: nHd.2.3.scaf00209
Output file: scaf209_pam_sites.csv
Target: All contigs
Allow non-canonical starts?: False
Max G-bases per site: 15
Scan type: Exhaustive
Buffered scan: Yes
Test site uniqueness: Yes
Only unique sites: No
Processes: 1

2016-09-12 11:22:34,007 - ngg2 - INFO - 223 total sites found by exhaustive scanning
2016-09-12 11:22:34,007 - ngg2 - INFO - 170 unique sites (76.2%)


In [40]:
df_209 =  pd.read_csv('scaf209_pam_sites.csv')
df_209['Match_gRNA'] = map(lambda x: matchSeq(x,hdAct1),df_209['gRNA_Seq'])

df_209_u = df_209[df_209.Unique=='Yes']
df_209_u['Match_gRNA'] = map(lambda x: matchSeq(x,hdAct1),df_209_u['gRNA_Seq'])

print('Total Found',len(df_209[df_209['Match_gRNA']==True]))
print('Unique Found',len(df_209_u[df_209_u['Match_gRNA']==True]))


df_209_u[df_209_u['Match_gRNA']]

('Total Found', 0)
('Unique Found', 0)


,Contig,Start,End,gRNA_Seq,PAM,Strand,G_start,Unique,Match_gRNA


In [37]:
# !ngg2 --outputFile scaf20_pam_sites.csv nHd.2.3.scaf00020
!ngg2 --outputFile scaf20_pam_sites.csv --allowNoncanonical nHd.2.3.scaf00020

2016-09-12 11:22:12,208 - ngg2 - INFO - ngg2 vv1.3.0

Options
FASTA: nHd.2.3.scaf00020
Output file: scaf20_pam_sites.csv
Target: All contigs
Allow non-canonical starts?: True
Max G-bases per site: 15
Scan type: Exhaustive
Buffered scan: Yes
Test site uniqueness: Yes
Only unique sites: No
Processes: 1

2016-09-12 11:22:12,323 - ngg2 - INFO - 1877 total sites found by exhaustive scanning
2016-09-12 11:22:12,323 - ngg2 - INFO - 1412 unique sites (75.2%)


In [41]:
df_20 =  pd.read_csv('scaf20_pam_sites.csv')
df_20['Match_gRNA'] = map(lambda x: matchSeq(x,hdMag1),df_20['gRNA_Seq'])

df_20_u = df_20[df_20.Unique=='Yes']
df_20_u['Match_gRNA'] = map(lambda x: matchSeq(x,hdMag1),df_20_u['gRNA_Seq'])

print('Total Found',len(df_20[df_20['Match_gRNA']==True]))
print('Unique Found',len(df_20_u[df_20_u['Match_gRNA']==True]))


df_20_u[df_20_u['Match_gRNA']]

('Total Found', 2)
('Unique Found', 2)


,Contig,Start,End,gRNA_Seq,PAM,Strand,G_start,Unique,Match_gRNA
848,nHd.2.3.scaf00020,218115,218134,CGACACGATGATCCGCAAGG,AGG,+,False,Yes,True
849,nHd.2.3.scaf00020,218118,218137,CACGATGATCCGCAAGGAGG,TGG,+,False,Yes,True
